# Homework 1. Programming in Racket

**Ilias Dzhabbarov**

### 1.1 Pseudo-Random Generation


In [46]:
(require racket)

(define a 1664525)
(define c 1013904223)
(define m (expt 2 32))

(struct gen (state))

(displayln "Exercise 1.1.1")

(define (mk-gen seed)
  (gen seed))

(define (next-gen g)
    (gen (remainder (+ c (* a (gen-state g))) m))
)

(println (mk-gen 123)) 
; #<gen>

(println (next-gen (mk-gen 123)))
; #<gen> 

(displayln "Exercise 1.1.2")

(define ((gen-integer i j) g)
    (cons 
        (+ i (remainder (gen-state g) (- j i)))
        (next-gen g)
    )
)

(println ((gen-integer 10 34) (mk-gen 123)))
; '(13 . #<gen>)
(println ((gen-integer 10 34) (mk-gen 123)))
; '(13 . #<gen>)    the same as above

(println ((gen-integer 10 34) 
            (next-gen (mk-gen 123))
           )
)
; '(32 . #<gen>)
(println ((gen-integer 10 34) 
            (cdr ((gen-integer 10 34) (mk-gen 123)))
            )
)
; '(32 . #<gen>)    the same as above



Exercise 1.1.1
#<gen>
#<gen>
Exercise 1.1.2
'(13 . #<gen>)
'(13 . #<gen>)
'(32 . #<gen>)
'(32 . #<gen>)


### Exercise 1.1.3
*previous cell required*

In [47]:
(displayln "Exercise 1.1.3")
(require racket/stream)

(define (stream-integers n m g)
    (stream-cons 
    (car ((gen-integer n m) g))
    (stream-integers n m (next-gen g))
    )
)

(println (stream->list (stream-take (stream-integers 1 10 (mk-gen 123)) 5)))
; (7 2 7 7 4)
(println (stream->list (stream-take (stream-integers 1 10 (mk-gen 123)) 10)))
; (7 2 7 7 4 1 8 4 9 6)     the first 5 integers the same as above


(displayln "Exercise 1.1.4")

(define (stream-random f g)
    (stream-cons 
    (car (f g))
    (stream-random f (cdr (f g))) ; not (next-gen g) because f could use generator more than once 
    )
)

(println (stream->list (stream-take (stream-random (gen-integer 1 10) (mk-gen 123)) 5)))
; (7 2 7 7 4)
(println (stream->list (stream-take (stream-random (gen-integer 1 10) (mk-gen 123)) 10)))
;(7 2 7 7 4 1 8 4 9 6)

Exercise 1.1.3
'(7 2 7 7 4)
'(7 2 7 7 4 1 8 4 9 6)
Exercise 1.1.4
'(7 2 7 7 4)
'(7 2 7 7 4 1 8 4 9 6)


### 1.2 Generating User-Defined Structures
*all previous cells are required*

In [48]:
(displayln "Exercise 1.2.1")
(define (gen-bool g)
    (cons 
        (zero? (remainder (gen-state g) 2))
        (next-gen g)
    )
)

(println (stream->list (stream-take (stream-random gen-bool (mk-gen 13123)) 5)))
; '(#f #t #f #t #f)

(displayln "Exercise 1.2.2")

(define ((gen-element lst) g)
    (cons
        (list-ref lst
            (car ((gen-integer 0 (length lst)) g))
        )
        (next-gen g)
    )
)

(println (stream->list (stream-take (stream-random (gen-element '(1 2 3)) (mk-gen 13)) 5)))
; '(2 1 3 1 1)

Exercise 1.2.1
'(#f #t #f #t #f)
Exercise 1.2.2
'(2 1 3 1 1)


In [49]:
; *all previous cells are required*

(displayln "Exercise 1.2.3")

(define ((gen-pair f1 f2) g)
    (cons 
        (cons 
            (car (f1 g))
            (car (f2 (cdr (f1 g))))
        )
        (cdr (f2 (cdr (f1 g))))
    )
)

(println (stream->list (stream-take (stream-random (gen-pair gen-bool gen-bool) (mk-gen 31337)) 3)))
; '((#f . #t) (#f . #t) (#f . #t))

(stream->list (stream-take (stream-random (gen-pair (gen-integer 1 10) (gen-integer 1 10)) (mk-gen 31337)) 3))
; '((9 . 3) (1 . 5) (8 . 8))

Exercise 1.2.3
'((#f . #t) (#f . #t) (#f . #t))


'((9 . 3) (1 . 5) (8 . 8))

In [50]:
(displayln "Exercise 1.2.4")


(define (stream-random-full f g)
    (stream-cons 
    (cons (car (f g)) (cdr (f g)))
    (stream-random-full f (cdr (f g))) 
    ;                     ^^^^^^^^^^^ 
    ; not (next-gen g) because `f` could use
    ; generator more than once 
    )
)

(define ((gen-list f n) g)
    (let 
        ([lst (stream->list (stream-take (stream-random-full f g) n))])
        
    (cons
        (for/list ([pair lst])
            (car pair)
        )
        (cdr (list-ref lst (- n 1)))
    )
    )
)
; '(7 2 7 7 4 1 8 4 9 6)


(println ((gen-list (gen-integer 1 10) 10) (mk-gen 123)))

(stream->list (stream-take (stream-random (gen-integer 1 10) (mk-gen 123)) 10))



Exercise 1.2.4
'((7 2 7 7 4 1 8 4 9 6) . #<gen>)


'(7 2 7 7 4 1 8 4 9 6)

In [51]:
(displayln "Exercise 1.2.5")

(define ((gen* . fs) g)
    (let-values (
        [
            (result final-g)
            (for/fold ([result empty]
                    [g g])  ; local vars
                    ([f fs])
            (values 
            (append result (list (car (f g))))
            (cdr (f g))))
        ]
    )
    (cons result final-g))
)




(println ((gen*) (mk-gen 123)))
; '(() . #<gen>)

(println ((gen* (gen-integer 1 10)) (mk-gen 123)))
; '((7) . #<gen>)

(println ((gen* (gen-integer 1 10) (gen-integer 10 20)) (mk-gen 123)))
; '((7 18) . #<gen>)


Exercise 1.2.5
'(() . #<gen>)
'((7) . #<gen>)
'((7 18) . #<gen>)


In [52]:
(displayln "Exercise 1.2.6")

(define ((gen-struct user-struct . fs) g)
    (cons
        (apply user-struct (car ((apply gen* fs) g)))
        (cdr ((apply gen* fs) g))
    )
)

(struct person (name age))
(let [(p ((gen-struct person
(gen-element '(Anna Boris Charlie))
(gen-integer 20 30))
(mk-gen 123)))]
(displayln (person-name (car p)))
(displayln (person-age (car p)))
(displayln (cdr p)))


Exercise 1.2.6
Anna
28
#<gen>


In [53]:
(displayln "Exercise 1.2.7")

(define (gen-int->int g)
    (cons

        (lambda (n)
            (+ 
                (* n 
                    (car ((gen-integer 0 10) g))
                ) 
                (car ((gen-integer 0 10) (next-gen g)))
            )
        )
        (next-gen (next-gen g))
    )
)
(map (lambda (f) (f 5))
(stream->list (stream-take (stream-random gen-int->int (mk-gen 4)) 10)))


Exercise 1.2.7


'(23 17 43 45 5 49 49 9 19 47)

### 1.3 Checking User-Defined Properties


In [54]:
(struct property (generator predicate))

(displayln "Exercise 1.3.1")


(define (stream-counterexamples-helper property g n)
    (cond 
        [(= n 0) empty-stream]
        [
            (not 
                ((property-predicate property) 
                 (car ((property-generator property) g)))
            )
            (stream-cons
                (car ((property-generator property) g))
                (stream-counterexamples-helper property (next-gen g) (- n 1))
            )
        ]
        [else 
            (stream-counterexamples-helper property (next-gen g) (- n 1))
        ]
    )
)

(define (stream-counterexamples property g)
    (stream-counterexamples-helper property g 100)
)




(println (stream⇒list 
    (stream-counterexamples (property (gen-integer 1 100) (lambda (x) (> x 5)))
(mk-gen 123))))
; '(2 1 4 3)


(println (stream->list (stream-counterexamples
(property (gen-integer 1 100)
(lambda (x) (> x 0)))
(mk-gen 123))))
; '()

Exercise 1.3.1


stream⇒list: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level
   C:\Program Files\Racket\share\pkgs\sandbox-lib\racket\sandbox.rkt:710:9: loop


In [55]:
(displayln "Exercise 1.3.2")

(define (stream-counterexample property g)
    (let 
        ([lst (stream->list (stream-counterexamples-helper property g 100))])
    (cond 
        [(empty? lst) empty-stream]
        [else (stream-cons (first lst) empty-stream)]
    )
    )
)

(println (stream->list 
    (stream-counterexample (property (gen-integer 1 100) (lambda (x) (> x 5)))
(mk-gen 123))))
; '(2 1 4 3)


(println (stream->list (stream-counterexample
(property (gen-integer 1 100)
(lambda (x) (> x 0)))
(mk-gen 123))))
; '()


Exercise 1.3.2
'(2)
'()


In [56]:
(displayln "Exercise 1.3.3")

(define (check property) 
    (let ([lst (stream->list (stream-counterexample property (mk-gen (current-seconds))))])
     (if (empty? lst)
         (displayln "Success")
         (values
            (display "Falsified!\nCounterexample: " )
            (display (first lst))
         
         )
     )
    )
)

Exercise 1.3.3


In [57]:
(check (property (gen-integer 1 10) (lambda (n) (< n 11))))


Success


In [58]:
(check (property (gen-integer 1 10) (lambda (n) (< n 5))))
; Falsified!
; Counterexample: 5


Falsified!
Counterexample: 6

In [59]:
(check (property
(gen-pair (gen-integer 1 10) (gen-integer 11 20))
(lambda (xy)
(let [(x (car xy))
(y (cdr xy))]
(< x y (+ x y))))))
; Success

Success


In [60]:
(check (property
(gen-pair (gen-integer 1 10) (gen-integer 11 20))
(lambda (xy)
(let [(x (car xy))
(y (cdr xy))]
(< x (- y x))))))
; Falsified!
; Counterexample: (8 . 15)

Falsified!
Counterexample: (7 . 14)

### 1.4 Implementing Properties


In [61]:
(displayln "Exercise 1.4.1")

(define (gen-2D-point g)
    (cons 
        (cons (car ((gen-integer -10 10) g)) (car ((gen-integer -10 10) (next-gen g))))
        (next-gen (next-gen g))
    )
)

(println (stream->list (stream-take (stream-random gen-2D-point (mk-gen 123)) 5)))
; '((-7 . 8) (-9 . -2) (-3 . 8) (3 . 2) (-3 . 8))

Exercise 1.4.1
'((-7 . 8) (-9 . -2) (-3 . 8) (3 . 2) (-3 . 8))


In [62]:
(displayln "Exercise 1.4.2")

(define (euclidean-distance-2D point1 point2)
    (let ([x1 (car point1)]
          [y1 (cdr point1)]
          [x2 (car point2)]
          [y2 (cdr point2)])
    (sqrt (+ (sqr (- x1 x2)) (sqr (- y1 y2)))))
) 

(euclidean-distance-2D '(3 . 4) '(0 . 0))
; 5

Exercise 1.4.2


5

In [63]:
(displayln "Exercise 1.4.3")
(define epsln 1e-5) ; accounting for error

(define (triangular gen mu)
    (property
        (gen-list gen 3)           ; generator
        (lambda (pts)              ; predicate
            (let ([p1 (car pts)]          
                  [p2 (cadr pts)]         
                  [p3 (caddr pts)])       
            (>= (+ (mu p1 p2) (mu p2 p3)) (- (mu p1 p3) epsln) )
            )
        )
    ) 
)

Exercise 1.4.3


In [64]:
(check (triangular gen-2D-point euclidean-distance-2D))
; Success

Success


In [65]:
(define (bad-distance-2D p1 p2)
    (let [(x1 (car p1))
        (x2 (car p2))
        (y1 (cdr p1))
        (y2 (cdr p2))]
        
    (+ x1 x2 y1 y2)))


(check (triangular gen-2D-point bad-distance-2D))
; Falsified!
; Counterexample: ((4 . 7) (-6 . -7) (4 . 3))

Falsified!
Counterexample: ((1 . 8) (-5 . -10) (1 . -8))

In [66]:
(displayln "Exercise 1.4.4")

(define (functor-law-1 map)
    (property
        (gen-list (gen-integer 1 10) (car ((gen-integer 10 100) (mk-gen (current-seconds)))))      ; generator
        (lambda (lst) (equal? (map identity lst) lst)) ; predicate
    )
)

Exercise 1.4.4


In [67]:
(check (functor-law-1 map))
; Success

Success


In [68]:
(define (bad-map f xs) (reverse xs))
(check (functor-law-1 bad-map))
; Falsified!
; Counterexample: (5 2 1 1 9 9 5 1 6 2 8 1 5 7 1 4 9 7 5 4 2 1 3)

Falsified!
Counterexample: (9 4 7 3 9 4 2 8 2 9 4 5 7 1 4 2 5 5)

In [69]:
(displayln "Exercise 1.4.5 (+0.25% extra credit)")


(define (functor-law-2 map)
    
    (
        let* (
            [gn (mk-gen (current-seconds))]
            [f (car (gen-int->int gn))]
            [g (car (gen-int->int (next-gen gn)))]
            ) 
    (property
        (gen-list (gen-integer 1 10) (car ((gen-integer 10 100) (next-gen (next-gen gn)))))      ; generator
        (lambda (lst) (equal? (map f (map g lst)) (map (lambda (x) (f (g x))) lst))) ; predicate
    )
    )
)

Exercise 1.4.5 (+0.25% extra credit)


In [70]:
(check (functor-law-2 map))
; Success

Success


In [71]:
(check (functor-law-2 bad-map))
; Falsified!
; Counterexample: (6 7 9 1 5 5 2 6 9 7 3 5 1 7 8 2 4 9 7 4 6 3 5 2 6 6 6 1 6 6 7 2 3 3 1 4)

Falsified!
Counterexample: (2 8 3 3 7 5 8 6 7 3 4 9 3 2 6 5 1 5 9 3 5 7 1 7 5 9 1 9 2 9 9 4 3 9 8 6 6 1 6 7 8 5 8 3 2 2 5 9 7 5 8 1 1 9 3 1 2 1 9 2 7 1 2 5 7 9)

In [72]:
(displayln "Exercise 1.4.6 (+0.25% extra credit)")

(define (f->predicate f)
  (lambda (x)
    (positive? (f x)))
)


(define (filter-shortens-list filter)
    (
    let* (
        [gn (mk-gen (current-seconds))]
        ) 
    (property
        (gen-pair 
            (gen-list 
                (gen-integer 1 10) (car ((gen-integer 10 100) gn))
            ) ; list
            gen-int->int ; predicate
        )  ; property-generator
        (lambda (lst) 
            (
            let (
                [xs (car lst)]
                [p (f->predicate (cdr lst))]
                )
            (<= (length (filter p xs)) (length xs))
            )
        ) ; property-predicate
    )
    )
)

Exercise 1.4.6 (+0.25% extra credit)


In [73]:
(check (filter-shortens-list filter))
; Success

Success


In [74]:
(define (bad-filter p lst)
(append (filter p lst) (filter p lst)))
(check (filter-shortens-list bad-filter))
; Falsified!
; Counterexample: ((9 9 6 4 5 6 5 9 6 4 7 2 7 1 1 7 5 3 5 4 3 7 7 9 8 1 4 2 1 8 7 6 9 5 6 7 3 1 7 8 2 4 8 3 7 2 7 6 3 9 4 8 3 2 8 9 5 5 8 5 3 3 6 9 2 9 9 7 6 8 5 6 4 3 4 3 4 7 6 5 4) . #<procedure>)


Falsified!
Counterexample: ((7 9 7 6 2 1 8 3 1 4 4 3 5 9 1 4 2 1 1 2 8 5 3 5 4) . #<procedure>)

### 1.5 Proving Properties

#### Exercise 1.5.1 (5 points).

Prove the first functor law for `map` and lists. Specifically, provide a correct definition of `map` and show that for any list `lst`, you have

```(map identity lst) = ... = lst```



In [75]:
(define (map f lst)
(
  if (null? lst)
    '()
    (cons 
      (f (car lst)) 
      (map f (cdr lst))
    )
)
)
; note:
; after this cell built-in map function
; will not be in the scope

Let's test my-map `map`

In [76]:
(check (functor-law-1 map))

Success


In [77]:
(check (functor-law-2 map))

Success


Let's use mathematical induction on the structure of the list `lst`.

### Base case

In [78]:
(define lst '())

In [79]:
(map identity '())

'()

In [80]:
(
  if (null? lst)
    '()
    (cons 
      (f (car lst)) 
      (map f (cdr lst))
    )
)

'()

In [81]:
(
  if (null? '())
    '()
    (cons 
      (identity (car '())) 
      (map identity (cdr '()))
    )
)

'()

In [82]:
(
  if #t
    '()
    (cons 
      (identity (car '())) 
      (map identity (cdr '()))
    )
)

'()

In [83]:
'()

'()

In [84]:
(map identity '()) = '()

'()

Q.E.D (base)

### Inductive Step:

Assume that for a list `xs`:

```racket
(map identity xs) = xs
```


Now, consider a list `lst` with one more element:

```racket
lst = (cons x xs)
```

Let's compute 

```racket
(map identity (cons x xs))
```
⇒

```racket
(
  if (null? lst)
    '()
    (cons 
      (f (car lst)) 
      (map f (cdr lst))
    )
)
```
⇒

```racket
(
  if (null? (cons x xs))
    '()
    (cons 
      (identity (car (cons x xs))) 
      (map identity (cdr (cons x xs)))
    )
)
```
⇒

```racket
(
  if (#f)
    '()
    (cons 
      (identity (car (cons x xs))) 
      (map identity (cdr (cons x xs)))
    )
)
```

⇒


```racket
(cons 
  (identity (car (cons x xs))) 
  (map identity (cdr (cons x xs)))
)
```

⇒

```racket
(cons 
  (identity x) 
  (map identity (cdr (cons x xs)))
)
```

⇒

```racket
(cons 
  x 
  (map identity (cdr (cons x xs)))
)
```

⇒

```racket
(cons 
  x 
  (map identity xs)
)
```

By the fisrt step

```racket
(map identity xs) = xs
```


So,

```racket
(cons 
  x 
  xs
)
```



In other words,

```racket

(map identity (cons x xs)) = (cons x xs)
```

Q.E.D (induction)

So, in general

```racket
(map identity lst) = lst
```

### Q.E.D

#### Exercise 1.5.2 (5 points). 

Prove the second functor law for `map` and lists. Specifically, provide a
correct definition of `map` and show that for any *pure* functions `f` and `g` and any list `lst`, you have


```racket
(map f (map g lst)) = ... = (map (lambda (x) (f (g x))) lst)
```

Let's review map definition:

In [85]:
(define (map f lst)
(
  if (null? lst)
    '()
    (cons 
      (f (car lst)) 
      (map f (cdr lst))
    )
)
)

#### Base case

##### 1.

```racket
(define lst '())
```

Let's compute and proof our equation


```racket
(map f (map g '()))
```
⇒

```racket
(map f
    (
    if (null? '())
        '()
        (cons 
        (g (car '())) 
        (map g (cdr '()))
        )
    )
)
```
⇒

```racket
(map f
    (
    if (#t)
        '()
        (cons 
        (g (car '())) 
        (map g (cdr '()))
        )
    )
)
```
⇒

```racket
(map f
    '() 
)
```
⇒

```racket
(
  if (null? '())
    '()
    (cons 
      (f (car '())) 
      (map f (cdr '()))
    )
)

```
⇒

```racket
(
  if (#t)
    '()
    (cons 
      (f (car '())) 
      (map f (cdr '()))
    )
)

```
⇒

```racket
'()

```

##### 2.

now let's compute

```racket
(map (lambda (x) (f (g x))) '())
```
⇒

```racket
(
  if (null? '())
    '()
    (cons 
      ((lambda (x) (f (g x))) (car lst)) 
      (map (lambda (x) (f (g x))) (cdr lst))
    )
)
```
⇒

```racket
(
  if (#t)
    '()
    (cons 
      ((lambda (x) (f (g x))) (car lst)) 
      (map (lambda (x) (f (g x))) (cdr lst))
    )
)
```
⇒

```racket

'()
```


now let's compute

```racket
(map (lambda (x) (f (g x))) '())
```
⇒

```racket
(
  if (null? '())
    '()
    (cons 
      ((lambda (x) (f (g x))) (car lst)) 
      (map (lambda (x) (f (g x))) (cdr lst))
    )
)
```
⇒

```racket
(
  if (#t)
    '()
    (cons 
      ((lambda (x) (f (g x))) (car lst)) 
      (map (lambda (x) (f (g x))) (cdr lst))
    )
)
```
⇒

```racket
'()
```

So we have:

```racket
(map f (map g '())) = '() = (map (lambda (x) (f (g x))) '())
```

#### Base case Q.E.D

#### Inductive Step:

Assume that for a list `xs`, the following holds:

```racket
(map f (map g xs)) = (map (lambda (x) (f (g x))) xs)
```

consider a list lst defined as:

```racket
(define lst (cons x xs))
```




##### 1

Let's compute

```racket
(map f (map g lst))
```
Or, in other terms:

```racket
(map f (map g (cons x xs)))
```
⇒

```racket
(map f 
    (
    if (null? (cons x xs))
        '()
        (cons 
        (g (car (cons x xs))) 
        (map g (cdr (cons x xs)))
        )
    )
)
```
⇒

```racket
(map f 
    (
    if (#f)
        '()
        (cons 
        (g (car (cons x xs))) 
        (map g (cdr (cons x xs)))
        )
    )
)
```
⇒

```racket
(map f 
    (cons 
    (g (car (cons x xs))) 
    (map g (cdr (cons x xs)))
    )
)
```
⇒

```racket
(map f 
    (cons 
    (g x) 
    (map g (cdr (cons x xs)))
    )
)
```
⇒

```racket
(map f 
    (cons 
    (g x) 
    (map g (cdr xs))
    )
)
```
⇒

```racket
(
  if (null? 
            (cons 
                (g x) 
                (map g (cdr xs))
            )
    )

    '()
    (cons 
      (f (car 
            (cons 
                (g x) 
                (map g (cdr xs))
            )
        )
      ) 
      (map f (cdr

                (cons 
                    (g x) 
                    (map g (cdr xs))
                )
            
             )
      )
    )
)
```
⇒



```racket
(
  if #f
    '()
    (cons 
      (f (car 
            (cons 
                (g x) 
                (map g (cdr xs))
            )
        )
      ) 
      (map f (cdr

                (cons 
                    (g x) 
                    (map g (cdr xs))
                )
            
             )
      )
    )
)
```
⇒


```racket

(cons 
    (f (car 
        (cons 
            (g x) 
            (map g (cdr xs))
        )
    )
    ) 
    (map f (cdr

            (cons 
                (g x) 
                (map g (cdr xs))
            )
        
            )
    )
)

```
⇒


```racket

(cons 
    (f (g x) ) 
    (map f (cdr

            (cons 
                (g x) 
                (map g (cdr xs))
            )
        
            )
    )
)

```
⇒
```racket

(cons 
    (f (g x) ) 
    (map f (map g (cdr xs)))
)



#### **Result**
```racket
(map f (map g (cons x xs))) 

= 

(cons 
    (f (g x) ) 
    (map f (map g (cdr xs)))
)
```

##### 2. 
Let's compute

```racket
(map (lambda (x) (f (g x))) (cons x xs))
```
⇒

```racket
(
  if (null? (cons x xs))
    '()
    (cons 
      ((lambda (x) (f (g x))) (car (cons x xs))) 
      (map (lambda (x) (f (g x))) (cdr (cons x xs)))
    )
)
```
⇒

```racket
(
  if (#f)
    '()
    (cons 
      ((lambda (x) (f (g x))) (car (cons x xs))) 
      (map (lambda (x) (f (g x))) (cdr (cons x xs)))
    )
)
```
⇒

```racket
(cons 
    ((lambda (x) (f (g x))) x) 
    (map (lambda (x) (f (g x))) (cdr (cons x xs)))
)
```
⇒

```racket
(cons 
    ((lambda (x) (f (g x))) x) 
    (map (lambda (x) (f (g x))) xs)
)
```
⇒

```racket
(cons 
    (f (g x)) 
    (map (lambda (x) (f (g x))) xs)
)
```

#### **Result**

```racket
(map (lambda (x) (f (g x))) (cons x xs))

=

(cons 
    (f (g x)) 
    (map (lambda (x) (f (g x))) xs)
)

```

By our inductive statement:

```racket
(map f (map g xs)) = (map (lambda (x) (f (g x))) xs)
```

So


```racket
(map (lambda (x) (f (g x))) (cons x xs))

=

(cons 
    (f (g x)) 
    (map (lambda (x) (f (g x))) xs)
)

=

(cons 
    (f (g x) ) 
    (map f (map g (cdr xs)))
)

=

(map f (map g (cons x xs))) 

```

### **Final result**

For any list `lst` (eq. `(cons x xs)`) the condition is met:

```racket
(map f (map g lst)) = ... = (map (lambda (x) (f (g x))) lst)

```

### Q.E.D.

#### Exercise 1.5.3 (+0.25% extra credit).

Prove that filtering a list cannot increase its length. Specifically, provide a correct definition of `filter` and show that for any pure predicate `p` and any list `lst`, you have

```racket
(length (filter p lst)) <= (length lst)
```

In [86]:
(define (filter p lst)
(cond 
    [(empty? lst) empty]
    [(p (first lst)) 
        (cons 
            (first lst) 
            (filter p (rest lst))
        )
    ]
    [else 
        (filter p (rest lst))
    ]
)       
)

In [87]:
(check (filter-shortens-list filter))

Success


### Prove by induction
#### Base case
Let `lst` be the empty list `'()`.

```racket
(length (filter p '()))
```
⇒

```racket
(length 
    (cond 
        [(empty? '()) empty]
        [(p (first '())) 
            (cons 
                (first '()) 
                (filter p (rest '()))
            )
        ]
        [else 
            (filter p (rest '()))
        ]
    )   
)
```
⇒

```racket
(length 
    (cond 
        [#t empty]
        [(p (first '())) 
            (cons 
                (first '()) 
                (filter p (rest '()))
            )
        ]
        [else 
            (filter p (rest '()))
        ]
    )   
)
```
⇒

```racket
(length empty)
```
⇒

```racket
0
```

So, in other words:

```racket
(length (filter p '())) = 0 <= 0 = (length '())
```

#### Base case Q.E.D

Consider a `lst` defined by:

```racket
(define lst (cons x xs))
```

where `xs` is a list for which the condition is satisfied:

```racket
(length (filter p xs)) <= (length xs)
```

Let's compute!

```racket
(length (filter p (cons x xs)))
```

⇒

```racket
(length
    (cond 
        [(empty? lst) (cons x xs)]
        [(p (first (cons x xs))) 
            (cons 
                (first (cons x xs)) 
                (filter p (rest (cons x xs)))
            )
        ]
        [else 
            (filter p (rest (cons x xs)))
        ]
    ) 
)
```

⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [(p (first (cons x xs))) 
            (cons 
                (first (cons x xs)) 
                (filter p (rest (cons x xs)))
            )
        ]
        [else 
            (filter p (rest (cons x xs)))
        ]
    ) 
)
```


⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [(p x) 
            (cons 
                (first (cons x xs)) 
                (filter p (rest (cons x xs)))
            )
        ]
        [else 
            (filter p (rest (cons x xs)))
        ]
    ) 
)
```

⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [(p x) 
            (cons 
                x 
                (filter p (rest (cons x xs)))
            )
        ]
        [else 
            (filter p (rest (cons x xs)))
        ]
    ) 
)
```
⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [(p x) 
            (cons 
                x 
                (filter p xs)
            )
        ]
        [else 
            (filter p (rest (cons x xs)))
        ]
    ) 
)
```
⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [(p x) 
            (cons 
                x 
                (filter p xs)
            )
        ]
        [else 
            (filter p xs)
        ]
    ) 
)
```

So, based on whenever `p x` true of false we will get different results.
Let's consider 2 cases

#### Case 1: `(p x)` is `#t` 

⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [#t
            (cons 
                x 
                (filter p xs)
            )
        ]
        [else 
            (filter p xs)
        ]
    ) 
)
```

⇒

```racket
(length
    (cons 
        x 
        (filter p xs)
    )
)
```

Knowing that

```racket
(length (filter p xs)) <= (length xs)
```

It's crear

```racket
+ (length (filter p xs)) 1 <= + (length xs) 1
```

And right and left parts are exactly:

```racket
(length (cons x (filter p xs))) <= (length (cons x xs))
```

**Statement proved by induction if `(p x)` is `#t`**

#### Case 2: `(p x)` is `#f` 

⇒

```racket
(length
    (cond 
        [#f (cons x xs)]
        [#f
            (cons 
                x 
                (filter p xs)
            )
        ]
        [else 
            (filter p xs)
        ]
    ) 
)
```

⇒

```racket
(length
    (filter p xs)
)
```

It's obvious that

```racket
(length (filter p xs)) <= (length xs) <= (length (cons x xs))
```



**Statement proved by induction if `(p x)` is `#f`**

#### Final point

To sum up, for any list `lst` getted from `(cons x xs)` we determined:


```racket
(length (filter p lst)) <= (length lst)

```

### Q.E.D.